In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/15 18:03:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
print(spark.version)

3.5.8


In [5]:
df_sales = spark.read.parquet("/opt/airflow/data/silver/sales")
df_profiles = spark.read.parquet("/opt/airflow/data/gold/user_profiles")

In [7]:
target_sales = df_sales.filter(
                (F.col("product_name") == "TV") & 
                (F.month("purchase_date") == 9) & 
                (F.dayofmonth("purchase_date").between(1, 10))
                )


In [9]:
enriched_sales = target_sales.join(df_profiles, on="client_id", how="inner") 

In [11]:
result_df = enriched_sales.withColumn("age", 
                         F.year(F.col("purchase_date")) - F.year(F.col("birth_date"))
                         ).filter(F.col("age").between(20, 30))

result_df.groupBy("state").count().orderBy(F.desc("count")).show(1)

+-----+-----+
|state|count|
+-----+-----+
|Idaho|  179|
+-----+-----+
only showing top 1 row

